In [23]:
! pip install duckdb
! pip install IPython

Defaulting to user installation because normal site-packages is not writeable


In [24]:
import pandas as pd
import glob
import time
import duckdb
from IPython import get_ipython
 

conn = duckdb.connect() # create an in-memory database

In [25]:
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable


In [26]:
# with pandas
cur_time = time.time()
df = pd.concat([pd.read_csv(f) for f in glob.glob('*.csv')])
print(f"time: {(time.time() - cur_time)}")
print(df.head(10))

time: 0.5435762405395508
   Order ID                     Product  Quantity Ordered     Price  \
0    176558        USB-C Charging Cable                 2     11.95   
1    176559  Bose SoundSport Headphones                 1     99.99   
2    176560                Google Phone                 1       600   
3    176560            Wired Headphones                 1     11.99   
4    176561            Wired Headphones                 1     11.99   
5    176562        USB-C Charging Cable                 1     11.95   
6    176563  Bose SoundSport Headphones                 1     99.99   
7    176564        USB-C Charging Cable                 1     11.95   
8    176565          Macbook Pro Laptop                 1  1,700.00   
9    176566            Wired Headphones                 1     11.99   

   Order Date      Time                        Purchase Address  \
0  19-04-2019   8:46 AM            917 1st St, Dallas, TX 75001   
1  07-04-2019  10:30 PM       682 Chestnut St, Boston, MA 0

In [27]:
# with duckdb
cur_time = time.time()
df = conn.execute("""
	SELECT *
	FROM '*.csv'
	LIMIT 10
""").df()
print(f"time: {(time.time() - cur_time)}")
print(df)

time: 0.06060910224914551
   Order ID                     Product  Quantity Ordered     Price  \
0    176558        USB-C Charging Cable                 2     11.95   
1    176559  Bose SoundSport Headphones                 1     99.99   
2    176560                Google Phone                 1       600   
3    176560            Wired Headphones                 1     11.99   
4    176561            Wired Headphones                 1     11.99   
5    176562        USB-C Charging Cable                 1     11.95   
6    176563  Bose SoundSport Headphones                 1     99.99   
7    176564        USB-C Charging Cable                 1     11.95   
8    176565          Macbook Pro Laptop                 1  1,700.00   
9    176566            Wired Headphones                 1     11.99   

  Order Date      Time                        Purchase Address  \
0 2019-04-19   8:46 AM            917 1st St, Dallas, TX 75001   
1 2019-04-07  10:30 PM       682 Chestnut St, Boston, MA 022

In [28]:
df = conn.execute("""
	SELECT *
	FROM '*.csv'
""").df()
conn.register("df_view", df)
conn.execute("DESCRIBE df_view").df() # doesn't work if you don't register df as a virtual table

,column_name,column_type,null,key,default,extra
0,Order ID,BIGINT,YES,None,None,None
1,Product,VARCHAR,YES,None,None,None
2,Quantity Ordered,BIGINT,YES,None,None,None
3,Price,VARCHAR,YES,None,None,None
4,Order Date,TIMESTAMP,YES,None,None,None
5,Time,VARCHAR,YES,None,None,None
6,Purchase Address,VARCHAR,YES,None,None,None
7,City,VARCHAR,YES,None,None,None
8,Product Type,VARCHAR,YES,None,None,None


In [29]:
conn.execute("SELECT COUNT(*) FROM df_view").df()

,count_star()
0,185950


In [30]:
df.isnull().sum()
df = df.dropna(how='all')

# Notice we use df and not df_view
# With DuckDB you can run SQL queries on top of Pandas dataframes
conn.execute("SELECT COUNT(*) FROM df").df()

,count_star()
0,185950


In [31]:
df.isnull().sum()
df = df.dropna(how='all')

# Notice we use df and not df_view
# With DuckDB you can run SQL queries on top of Pandas dataframes
conn.execute("SELECT COUNT(*) FROM df").df()

,count_star()
0,185950


In [32]:

conn.execute("""SELECT * FROM df WHERE "Order ID"='295665'""").df()


,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,295665,Macbook Pro Laptop,1,"1,700.00",2019-12-30,12:01 AM,"136 Church St, New York City, NY 10001",New York City,Laptop


In [37]:
print(df.dtypes)
df["Price"] = df["Price"].str.replace(",", "").astype(float)


Order ID                     int64
Product                     object
Quantity Ordered             int64
Price                       object
Order Date          datetime64[us]
Time                        object
Purchase Address            object
City                        object
Product Type                object
dtype: object


In [38]:
conn.execute("""
CREATE OR REPLACE TABLE sales AS
SELECT
    "Order ID"::INTEGER AS order_id,
    Product AS product,
    "Quantity Ordered"::INTEGER AS quantity,
    "Price"::DECIMAL AS price_each,
    "Order Date"::DATE AS order_date,
    "Purchase Address" AS purchase_address
FROM df
WHERE
    TRY_CAST("Order ID" AS INTEGER) NOTNULL

""")

In [39]:
# Fetch all rows from the 'sales' table
sales_data = conn.execute("SELECT * FROM sales").fetchdf()
print(sales_data)

        order_id                     product  quantity  price_each order_date  \
0         176558        USB-C Charging Cable         2       11.95 2019-04-19   
1         176559  Bose SoundSport Headphones         1       99.99 2019-04-07   
2         176560                Google Phone         1      600.00 2019-04-12   
3         176560            Wired Headphones         1       11.99 2019-04-12   
4         176561            Wired Headphones         1       11.99 2019-04-30   
...          ...                         ...       ...         ...        ...   
185945    259353      AAA Batteries (4-pack)         3        2.99 2019-09-17   
185946    259354                      iPhone         1      700.00 2019-09-01   
185947    259355                      iPhone         1      700.00 2019-09-23   
185948    259356      34in Ultrawide Monitor         1      379.99 2019-09-19   
185949    259357        USB-C Charging Cable         1       11.95 2019-09-30   

                           